# Better methods for patch offset representation

### Checklist
1. [x] Initial layout on colmap cloud (no match solution)
2. [x] Generate representative figure
3. [x] Layout on satellite image
4. [x] Individual solutions
5. [x] Representative figure
6. [ ] Joint overlay

In [1]:
import numpy as np
import cv2
import open3d as o3d
import plotly.graph_objects as go
import plotly.io as pio
from scipy.spatial.transform import Rotation as R
from scipy.spatial import cKDTree
# %matplotlib qt
import matplotlib.pyplot as plt
from matplotlib.path import Path
import time
import imageio

from groundNAV_agent import *
from colmapParsingUtils import *
import pycolmap

# SAVE YOUR WORK
%load_ext autoreload
%autoreload 2
%autosave 180

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Autosaving every 180 seconds


In [2]:
# Load in necessary parameters for gNAV agent 
# Define Class Parameters 

images_colm  = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/images.txt"
cameras_colm = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/cameras.txt"
pts3d_colm = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/points3D_f.txt"

# Images selected for local corrections
image_1 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9475.JPEG"
image_2 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9464.JPEG"
image_3 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9467.JPEG"
image_4 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9473.JPEG"
image_5 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9476.JPEG"
image_6 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9446.JPEG" #ID:4
image_7 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9520.JPEG" #ID:78
image_8 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9531.JPEG" #ID:89
image_9 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9542.JPEG" #ID:100
image_10 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9576.JPEG"
# Load in satellite reference image
sat_ref = "TTurf/TurfSat.jpg"

# Organize for agent params
images = [image_1, image_2, image_3, image_4, image_5, image_6, image_7, image_8, image_9, image_10]

In [3]:
# Create class
gnav = gNAV_agent(images_colm, cameras_colm, pts3d_colm, images, sat_ref)

# Grab raw points and RGB data for scene and reference cloud
scene_pts, rgb_data = gnav.grab_pts(gnav.pts3d_c)
ref_pts, ref_rgb = gnav.ref_pts, gnav.ref_rgb

In [4]:
# Use ground plane pts to set reference frame 
# Need gravity and height
pts_gnd_idx = np.array([25440, 25450, 25441, 25449, 25442, 25445, 103922, 103921, 103919, 103920])
# tform_ref_frame = gnav.set_ref_frame(pts_gnd_idx) # THIS IS WHAT I AM CHANGING 
tform_ref_frame = gnav.set_ref_frame_mid(pts_gnd_idx) # NEW VERSION
tform_ref_frame_pts = gnav.inv_homog_transform(tform_ref_frame)
print("\nReference frame transformation\n", tform_ref_frame_pts)


# Transform all points to the new coordinate system 
# Not necessary since we aren't using the cloud, but a good visual check for coord frame
tform_ref_frame_inv = gnav.inv_homog_transform(tform_ref_frame)
origin_ref, scene_pts_ref, scene_vec_ref = gnav.unit_vec_tform(scene_pts, gnav.origin_w, tform_ref_frame_inv)
# print(origin_ref)
# Transform scene cloud to 2D (also as a visual check)
# Note: 2d projection will look off with z=-1; see TTurf_v2 for cropping method
scene_ref_2d = gnav.proj_2d_scene(scene_pts_ref)
# print(scene_ref_2d)


Reference frame transformation
 [[-1.55069060e-03  9.81197008e-01  1.93002661e-01 -1.21025836e-01]
 [-1.42845166e-01 -1.91240997e-01  9.71093270e-01  1.86102525e+00]
 [ 9.89743833e-01 -2.60636319e-02  1.40455805e-01  7.28134156e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [5]:
# Image 0 - 10 yard hash marks
imnum = 0
x,y = 1500,1000
side_x = 900 # WIDTH
side_y = 500 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)

In [6]:
# Image 1 - bottom of S
imnum = 1
x,y = 1400,2300
side_x = 800 # WIDTH
side_y = 1200 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)

In [7]:
# Image 2 - Jumbo logo OR B in jumbos
imnum = 2
x,y = 800,2100
side_x = 1400 # WIDTH
side_y = 500 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)

In [8]:
# Image 3 - Edge to endzone 
imnum = 3
x,y = 1150,900
side_x = 1200 # WIDTH
side_y = 500 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)

In [9]:
# Image 4 - 10yd marker
imnum = 4
x,y = 1200,1300
side_x = 1000 # WIDTH
side_y = 700 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)

In [10]:
# Image 5 - Bottom half of S
imnum = 5
x,y = 150, 2000
side_x = 1500
side_y = 250 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)

In [11]:
# Image 6 - 40yd marker
imnum = 6
x,y = 1150,1150
side_x = 1100 # WIDTH
side_y = 900 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)

In [12]:
# Image 7 - Jumbo logo
imnum = 7
x,y = 400,1500 # 1700
side_x = 1500 # WIDTH
side_y = 750 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)

In [13]:
# Image 8 - 30yd marker (near)
imnum = 8
x,y = 1350,1450
side_x = 1200 # WIDTH
side_y = 500 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)

In [14]:
# Image 9 - bottom of U in Jumbos
imnum = 9
x,y = 600,1750
side_x = 1000 # WIDTH
side_y = 700 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)

## 1. Initial layout on colmap cloud (no match solution)
- [x] Generate projections
- [x] Mean location

In [15]:
## Generate projection of image sections 
for i in range(len(images)):
# Just for the first image for now
# for i in range(1):
    # Unit vectors in camera coords 
    pts_vec_c, pts_rgb_gnd = gnav.unit_vec_c(i)
    gnav.im_mosaic[i] = {'rgbc': pts_rgb_gnd}

    # Get transformation matrix that move from camera coords to world coords
    id = gnav.im_ids[i]
    homog_w2c, homog_c2w = gnav.get_pose_id(id,i)
    # print('Homogeneous transformation from world to camera \n', homog_c2w)
    # print('\n Homogeneous transformation from camera to world \n', homog_w2c)

    # Transform to world coords
    origin_c, pts_loc_w, pts_vec_w = gnav.unit_vec_tform(pts_vec_c, gnav.origin_w, homog_c2w)
    # print('\n New camera frame origin = ', origin_c)
    
    # Get new points 
    ranges, new_pts_w = gnav.pt_range(pts_vec_w, homog_c2w, origin_c, i)
    # print('\nNew Points \n', new_pts_w)

    # Transfer points to reference frame
    __, new_pts_r, pts_vec_r = gnav.unit_vec_tform(new_pts_w, gnav.origin_w, tform_ref_frame_pts)

    # Convert points to grayscale 
    gray_c = gnav.conv_to_gray(gnav.im_mosaic[i]['rgbc'],i)
    # print(gray_c)

    # Put new points and grayscale colors in image mosaic
    gnav.im_mosaic[i]['pts'] = new_pts_r
    gnav.im_mosaic[i]['color_g'] = gray_c
    
    print("\nDone image ", i)


Done image  0

Done image  1

Done image  2

Done image  3

Done image  4

Done image  5

Done image  6

Done image  7

Done image  8

Done image  9


In [16]:
# Grab mean point location on COLMAP scale 
means_patch_colm = np.zeros((len(images),3))
for i in range(len(images)):
    pts = gnav.im_mosaic[i]['pts'] # Grab pts
    # print(np.mean(pts, axis=0))
    means_patch_colm[i] = np.mean(pts,axis=0) # Calc mean
print("Mean location for each colmap patch: \n", means_patch_colm)

Mean location for each colmap patch: 
 [[-0.78040276  0.79359115  1.00111726]
 [-0.73513128 -1.00307527  1.00111726]
 [-1.72396002 -0.06408308  1.00111726]
 [-1.35868667  0.41415602  1.00111726]
 [ 0.76138545 -0.15151221  1.00111726]
 [-0.49264411 -2.32512668  1.00111726]
 [ 0.87608305  8.21747279  1.00111726]
 [ 4.29273404  6.25703641  1.00111726]
 [ 4.02119463  2.68858411  1.00111726]
 [-4.22097577  2.73338327  1.00111726]]


## 2. Representative Figure 
- [x] Circles to represent
- [x] Lines connecting circles
- [x] Plot

### NOTES
- The projections are not aligned with each other, but not on the ground plane (shouldn't matter)

In [17]:
# Generate a lineset
points = means_patch_colm
edges = np.array([[0,3],[3,2],[2,1],[1,5],[5,4],[4,8],[8,7],[7,6],[6,9],[9,0]])

# Create a LineSet object
lines = o3d.geometry.LineSet()

# Set points and edges
lines.points = o3d.utility.Vector3dVector(points)
lines.lines = o3d.utility.Vector2iVector(edges)

In [18]:
# # PLOT representative figure

# # Use open3d to create point cloud visualization 
# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Mosaic scene projection - COLMAP coords")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)


# # SPHERES
# for i in range(len(images)):
#     sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.25, resolution=20)
#     sphere.translate((means_patch_colm[i]))
#     sphere.paint_uniform_color([1.0, 0.3, 0.3])
#     vis.add_geometry(sphere)

# # LINES
# vis.add_geometry(lines)


# # POINT CLOUDS 
# for i in range(len(images)):
#     cloud = o3d.geometry.PointCloud()
#     cloud.points = o3d.utility.Vector3dVector(gnav.im_mosaic[i]['pts'])
#     cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[i]['color_g'])
#     vis.add_geometry(cloud)


# # vis.add_geometry(axis_origin)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # # Set up initial viewpoint
# # view_control = vis.get_view_control()
# # # Direction which the camera is looking
# # view_control.set_front([0, 0, -1])  # Set the camera facing direction
# # # Point which the camera revolves about 
# # view_control.set_lookat([1, 3.5, 1])   # Set the focus point
# # # Defines which way is up in the camera perspective 
# # view_control.set_up([-1, 1, 0])       # Set the up direction
# # view_control.set_zoom(.45)           # Adjust zoom if necessary


# # vis.poll_events()
# # vis.update_renderer()
# # # GRAB SCREEN CLIP
# # vis.capture_screen_image("Figs_ScrSh/Error_Rep/Colm/patches_circles_lines_colm.png")
# # time.sleep(1.5)  # Keep window open for 1.5 seconds

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

In [19]:
print(gnav.im_mosaic)

{0: {'rgbc': array([[0.3372549 , 0.56470588, 0.45490196],
       [0.36078431, 0.58823529, 0.47843137],
       [0.38823529, 0.61568627, 0.50588235],
       ...,
       [0.37647059, 0.63529412, 0.52156863],
       [0.39607843, 0.65490196, 0.54117647],
       [0.53333333, 0.79215686, 0.67843137]]), 'pts': array([[-0.5833712 ,  1.10825017,  1.00111726],
       [-0.58377642,  1.10827486,  1.00111726],
       [-0.58418161,  1.10829955,  1.00111726],
       ...,
       [-0.89728728,  0.57490911,  1.00111726],
       [-0.89751512,  0.57493573,  1.00111726],
       [-0.89774294,  0.57496234,  1.00111726]]), 'color_g': array([[0.48418039, 0.48418039, 0.48418039],
       [0.5077098 , 0.5077098 , 0.5077098 ],
       [0.53516078, 0.53516078, 0.53516078],
       ...,
       [0.54494118, 0.54494118, 0.54494118],
       [0.56454902, 0.56454902, 0.56454902],
       [0.70180392, 0.70180392, 0.70180392]])}, 1: {'rgbc': array([[0.04313725, 0.20392157, 0.09803922],
       [0.0627451 , 0.22352941, 0.1176470

## 3. Layout on satellite image

In [20]:
# Implementing an initial guess for the local image 

# ARBITRARY SCALE
# scale = 78
# yaw = 2.44777
# x = 0
# y = 0


# BEST after CONVERGENCE
scale = 80.14819958
yaw = 2.44777939
x = -55.46495001
y = 21.62910054


tform_guess = gnav.tform_create(x,y,0,0,0,yaw)
gnav.best_guess_tform = tform_guess
gnav.best_guess_scale = scale
# gnav.best_guess_scale_MD = s_mat # Multi-directional (MD)
# print(tform_guess)


# NEW IMPLEMENTATION STRATEGY
for i in range(len(images)):
# Just for the first image for now
# for i in range(1):
    loc_im_pts = gnav.im_mosaic[i]['pts'].copy()
    # print(loc_im_pts)

    loc_im_pts[:, :2] *= scale # For SINGLE scale factor
    # loc_im_pts = loc_im_pts @ s_mat # For MULTI-DIRECTIONAL scale factor
    # Get new points 
    __, loc_im_pts_guess, loc_im_vec_guess = gnav.unit_vec_tform(loc_im_pts, gnav.origin_w, tform_guess)
    gnav.im_pts_best_guess[i] = {'pts': loc_im_pts_guess}
    # gnav.im_pts_best_guess[i]['tree'] = cKDTree(loc_im_pts_guess) # UNECESSARY 

    print("\nDone image ", i)


Done image  0

Done image  1

Done image  2

Done image  3

Done image  4

Done image  5

Done image  6

Done image  7

Done image  8

Done image  9


In [21]:
# # PLOT representative figure

# # Use open3d to create point cloud visualization 
# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Mosaic scene projection - COLMAP coords")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)
# vis.add_geometry(axis_origin)

# # POINT CLOUDS 
# for i in range(len(images)):
#     cloud = o3d.geometry.PointCloud()
#     cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[i]['pts'])
#     cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[i]['color_g'])
#     vis.add_geometry(cloud)

# # Create point cloud for reference cloud (satellite)
# ref_cloud = o3d.geometry.PointCloud()
# ref_cloud.points = o3d.utility.Vector3dVector(gnav.ref_pts)
# ref_cloud.colors = o3d.utility.Vector3dVector(gnav.ref_rgb)
# vis.add_geometry(ref_cloud)


# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # # Set up initial viewpoint
# # view_control = vis.get_view_control()
# # # Direction which the camera is looking
# # view_control.set_front([0, .1, -1])  # Set the camera facing direction
# # # Point which the camera revolves about 
# # view_control.set_lookat([1, 3.5, 1])   # Set the focus point
# # # Defines which way is up in the camera perspective 
# # view_control.set_up([-1, 1, 0])       # Set the up direction
# # view_control.set_zoom(.45)           # Adjust zoom if necessary


# # vis.poll_events()
# # vis.update_renderer()
# # # GRAB SCREEN CLIP
# # vis.capture_screen_image("Figs_ScrSh/Error_Rep/circles_lines_colm.png")
# # time.sleep(1.5)  # Keep window open for 1.5 seconds

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

## 4. Individual Solutions

- saved as <'ind_params.npy'>

In [22]:
gnav.params = np.load('ind_params.npy')
print(gnav.params)

[[  82.            2.46091425  -56.           28.        ]
 [  82.            2.46091425  -60.           14.        ]
 [  76.            2.37364778  -44.           27.        ]
 [  78.            2.40855437  -52.           23.        ]
 [  80.            2.44346095  -51.           22.        ]
 [  87.            2.42600766  -84.          -24.        ]
 [  71.            2.42600766  -48.           42.        ]
 [  68.            2.58308729 -105.           67.        ]
 [  92.            2.35619449   96.            1.        ]
 [  85.            2.42600766  -79.          112.        ]]


In [23]:
# APPLY Individual Solutions

# Transform 
for i in range(len(images)):
# Just for the first image for now
# for i in range(1):
    loc_im_pts = gnav.im_mosaic[i]['pts'].copy()
    # print(loc_im_pts)
    # Grab parameters 
    p = gnav.params[i]
    scale, yaw, x, y = p[0], p[1], p[2], p[3]
    loc_im_pts[:, :2] *= scale # For SINGLE scale factor
    # loc_im_pts = loc_im_pts @ s_mat # For MULTI-DIRECTIONAL scale factor
    # Make SPECIFIC transformation matrix 
    tform_guess = gnav.tform_create(x,y,0,0,0,yaw)
    # Get new points 
    __, loc_im_pts_guess, loc_im_vec_guess = gnav.unit_vec_tform(loc_im_pts, gnav.origin_w, tform_guess)
    gnav.im_pts_best_guess[i] = {'pts': loc_im_pts_guess}
    # gnav.im_pts_best_guess[i]['tree'] = cKDTree(loc_im_pts_guess) # UNECESSARY 

    print("\nDone image ", i)


Done image  0

Done image  1

Done image  2

Done image  3

Done image  4

Done image  5

Done image  6

Done image  7

Done image  8

Done image  9


In [24]:
# # VISUALIZE

# # PLOT representative figure

# # Use open3d to create point cloud visualization 
# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Mosaic scene projection - COLMAP coords")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=40)
# vis.add_geometry(axis_origin)

# # POINT CLOUDS 
# for i in range(len(images)):
#     cloud = o3d.geometry.PointCloud()
#     cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[i]['pts'])
#     cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[i]['color_g'])
#     vis.add_geometry(cloud)

# # Create point cloud for reference cloud (satellite)
# ref_cloud = o3d.geometry.PointCloud()
# ref_cloud.points = o3d.utility.Vector3dVector(gnav.ref_pts)
# ref_cloud.colors = o3d.utility.Vector3dVector(gnav.ref_rgb)
# vis.add_geometry(ref_cloud)


# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

### 5. Representative Figure

In [25]:
# Grab mean point location on COLMAP scale 
means_patch_sat = np.zeros((len(images),3))
for i in range(len(images)):
    pts = gnav.im_pts_best_guess[i]['pts'] # Grab pts
    # print(np.mean(pts, axis=0))
    means_patch_sat[i] = np.mean(pts,axis=0) # Calc mean
print("Mean location for each colmap patch: \n", means_patch_sat)

Mean location for each colmap patch: 
 [[ -47.22077093  -62.84448125    1.00111726]
 [  38.60992186   39.98602945    1.00111726]
 [  53.63179651  -60.51139695    1.00111726]
 [   5.14096698  -71.91950513    1.00111726]
 [ -89.8691938    70.43793753    1.00111726]
 [  81.05845136  100.54848335    1.00111726]
 [-477.71582023 -357.52007041    1.00111726]
 [-578.01949705 -139.13964244    1.00111726]
 [-340.49676475   87.69141021    1.00111726]
 [  39.34975633 -298.7301833     1.00111726]]


In [26]:
# Generate a lineset
points = means_patch_sat
edges = np.array([[0,3],[3,2],[2,1],[1,5],[5,4],[4,8],[8,7],[7,6],[6,9],[9,0]])

# Create a LineSet object
lines_sat = o3d.geometry.LineSet()

# Set points and edges
lines_sat.points = o3d.utility.Vector3dVector(points)
lines_sat.lines = o3d.utility.Vector2iVector(edges)

In [27]:
# # PLOT representative figure

# # Use open3d to create point cloud visualization 
# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Mosaic scene projection - SATELLITE coords")


# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=40)
# # vis.add_geometry(axis_origin)

# # POINT CLOUDS 
# for i in range(len(images)):
#     cloud = o3d.geometry.PointCloud()
#     cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[i]['pts'])
#     cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[i]['color_g'])
#     # vis.add_geometry(cloud)

# # SPHERES
# for i in range(len(images)):
#     sphere = o3d.geometry.TriangleMesh.create_sphere(radius=10, resolution=20)
#     sphere.translate((means_patch_sat[i]))
#     sphere.paint_uniform_color([0.3, 1.0, 0.3])
#     vis.add_geometry(sphere)

# # LINES
# vis.add_geometry(lines_sat)

# # Create point cloud for reference cloud (satellite)
# ref_cloud = o3d.geometry.PointCloud()
# ref_cloud.points = o3d.utility.Vector3dVector(gnav.ref_pts)
# ref_cloud.colors = o3d.utility.Vector3dVector(gnav.ref_rgb)
# # vis.add_geometry(ref_cloud)


# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # # Set up initial viewpoint
# # view_control = vis.get_view_control()
# # # Direction which the camera is looking
# # view_control.set_front([0, .1, -1])  # Set the camera facing direction
# # # Point which the camera revolves about 
# # view_control.set_lookat([1, 3.5, 1])   # Set the focus point
# # # Defines which way is up in the camera perspective 
# # view_control.set_up([-1, 1, 0])       # Set the up direction
# # view_control.set_zoom(.45)           # Adjust zoom if necessary
# # GOOD ONE BELOW
# # # Set up initial viewpoint
# # view_control = vis.get_view_control()
# # # Direction which the camera is looking
# # view_control.set_front([0, 0.01, -1])  # Set the camera facing direction
# # # Point which the camera revolves about 
# # view_control.set_lookat([-200, -100, 0])   # Set the focus point
# # # Defines which way is up in the camera perspective 
# # view_control.set_up([0, -1, 0])       # Set the up direction
# # view_control.set_zoom(.45)           # Adjust zoom if necessary


# # vis.poll_events()
# # vis.update_renderer()
# # # GRAB SCREEN CLIP
# # vis.capture_screen_image("Figs_ScrSh/Error_Rep/circles_lines_sat.png")
# # time.sleep(1.5)  # Keep window open for 1.5 seconds

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

### 6. JOINT Overlay 

- [ ] Apply transformation to each section
- [ ] Get mean
- [ ] Align specific dots (bottom corners?)

In [28]:
# Apply transformation to all sections

# Solution to use
# BEST after CONVERGENCE
scale = 80.14819958
yaw = 2.44777939
x = -55.46495001
y = 21.62910054

# scale = 70
# yaw = 2.44777939
# x = -50
# y = 0

tform_guess = gnav.tform_create(x,y,0,0,0,yaw)
gnav.im_pts_S_colm = {}

# IMPLEMENTATION STRATEGY
for i in range(len(images)):
# Just for the first image for now
# for i in range(1):
    loc_im_pts = gnav.im_mosaic[i]['pts'].copy()
    # print(loc_im_pts)

    loc_im_pts[:, :2] *= scale # For SINGLE scale factor
    # loc_im_pts = loc_im_pts @ s_mat # For MULTI-DIRECTIONAL scale factor
    # Get new points 
    __, loc_im_pts_guess, loc_im_vec_guess = gnav.unit_vec_tform(loc_im_pts, gnav.origin_w, tform_guess)
    gnav.im_pts_S_colm[i] = {'pts': loc_im_pts_guess}
    # gnav.im_pts_best_guess[i]['tree'] = cKDTree(loc_im_pts_guess) # UNECESSARY 

    print("\nDone image ", i)

# Gather mean 
means_patch_colm_S = np.zeros((len(images),3))
for i in range(len(images)):
    pts = gnav.im_pts_S_colm[i]['pts'] # Grab pts
    # print(np.mean(pts, axis=0))
    means_patch_colm_S[i] = np.mean(pts,axis=0) # Calc mean
print("Mean location for each colmap patch: \n", means_patch_colm_S)


Done image  0

Done image  1

Done image  2

Done image  3

Done image  4

Done image  5

Done image  6

Done image  7

Done image  8

Done image  9
Mean location for each colmap patch: 
 [[ -48.05097127  -67.26889318    1.00111726]
 [  41.24336179   45.76015721    1.00111726]
 [  54.04814972  -62.77968532    1.00111726]
 [   7.02935379  -73.52706721    1.00111726]
 [ -94.61534885   69.9881519     1.00111726]
 [  94.06018672  139.65173732    1.00111726]
 [-530.61556337 -439.82167956    1.00111726]
 [-640.66857936 -143.90915374    1.00111726]
 [-441.04395667   62.05815165    1.00111726]
 [  64.534168   -363.13540704    1.00111726]]


In [29]:
# Generate a lineset
points = means_patch_colm_S
edges = np.array([[0,3],[3,2],[2,1],[1,5],[5,4],[4,8],[8,7],[7,6],[6,9],[9,0]])

# Create a LineSet object
lines_colm = o3d.geometry.LineSet()

# Set points and edges
lines_colm.points = o3d.utility.Vector3dVector(points)
lines_colm.lines = o3d.utility.Vector2iVector(edges)

In [30]:
# PLOT representative figure

# Use open3d to create point cloud visualization 
# Create visualization 
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Mosaic scene projection - COLMAP coords")

# Create axes @ origin
axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=40)


# COLMAP SOLUTION 
# SPHERES
for i in range(len(images)):
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=10, resolution=20)
    sphere.translate((means_patch_colm_S[i]))
    sphere.paint_uniform_color([1.0, 0.3, 0.3])
    vis.add_geometry(sphere)
# LINES
vis.add_geometry(lines_colm)
# POINT CLOUDS 
for i in range(len(images)):
    cloud = o3d.geometry.PointCloud()
    cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_S_colm[i]['pts'])
    cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[i]['color_g'])
    vis.add_geometry(cloud)

# SATELLITE SOLUTION
# POINT CLOUDS 
for i in range(len(images)):
    cloud = o3d.geometry.PointCloud()
    cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[i]['pts'])
    cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[i]['color_g'])
    vis.add_geometry(cloud)
# SPHERES
for i in range(len(images)):
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=10, resolution=20)
    sphere.translate((means_patch_sat[i]))
    sphere.paint_uniform_color([0.3, 1.0, 0.3])
    vis.add_geometry(sphere)
# LINES
vis.add_geometry(lines_sat)


# Create point cloud for reference cloud (satellite)
ref_cloud = o3d.geometry.PointCloud()
ref_cloud.points = o3d.utility.Vector3dVector(gnav.ref_pts)
ref_cloud.colors = o3d.utility.Vector3dVector(gnav.ref_rgb)
vis.add_geometry(ref_cloud)

# vis.add_geometry(axis_origin)

# # Size options (jupyter gives issues when running this multiple times, but it looks better)
# render_option = vis.get_render_option()
# render_option.point_size = 2

# Set up initial viewpoint
view_control = vis.get_view_control()
# Direction which the camera is looking
view_control.set_front([0, 0.03, -1])  # Set the camera facing direction
# Point which the camera revolves about 
view_control.set_lookat([-225, -125, 0])   # Set the focus point
# Defines which way is up in the camera perspective 
view_control.set_up([0, -1, 0])       # Set the up direction
view_control.set_zoom(.45)           # Adjust zoom if necessary


vis.poll_events()
vis.update_renderer()
# GRAB SCREEN CLIP
vis.capture_screen_image("Figs_ScrSh/Error_Rep/overlay_genSol_wSAT.png")
time.sleep(1.5)  # Keep window open for 1.5 seconds

# Run and destroy visualization 
# vis.run()
vis.destroy_window()

libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 74
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 74
pci id for fd 74: 8086:a7a0, driver iris
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using DRI3 for screen 0
